<a href="https://colab.research.google.com/github/AsmaSaiyed/Module21_deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [2]:
application_df.columns

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(columns=['EIN', 'NAME','STATUS'])
application_df.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0   Association              0                      N     5000              1  
1  Co-operative         1-9999                      N   108590              1  
2   Association              0                      N     5000              0  
3         Trust    10000-24999                      N     6692              1  
4         Trust  100000-499999                      N   142590              1

In [4]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
value_counts=application_df.value_counts("APPLICATION_TYPE")
value_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64

In [42]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
# cut off value is 500
cutoff = 500
application_types_to_replace = list(value_counts[value_counts < cutoff].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [41]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
value_count=application_df.value_counts("CLASSIFICATION")
value_count

CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
dtype: int64

In [43]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE

application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [45]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
cutoff = 1000
classifications_to_replace = list(value_count[value_count < cutoff].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [46]:
application_df.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0            Other       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          Other    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0   Association              0                      N     5000              1  
1  Co-operative         1-9999                      N   108590              1  
2   Association              0                      N     5000              0  
3         Trust    10000-24999                      N     6692              1  
4         Trust  100000-499999                      N   142590              1

In [47]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
new_df=pd.get_dummies(application_df, prefix=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE','ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'])
new_df.head()

ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T19  \
0     5000              1                       1                     0   
1   108590              1                       0                     0   
2     5000              0                       0                     0   
3     6692              1                       0                     0   
4   142590              1                       0                     0   

   APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  \
0                    0                    0                    0   
1                    1                    0                    0   
2                    0                    0                    1   
3                    1                    0                    0   
4                    1                    0                    0   

   APPLICATION_TYPE_T6  AFFILIATION_CompanySponsored  \
0                    0                             0   
1                    0                             0   
2                    0                             1   
3                    0                             1   
4                    0                             0   

   AFFILIATION_Family/Parent  ...  SPECIAL_CONSIDERATIONS_1-9999  \
0                          0  ...                              0   
1                          0  ...                              1   
2                          0  ...                              0   
3                          0  ...                              0   
4                          0  ...                              0   

   SPECIAL_CONSIDERATIONS_10000-24999  SPECIAL_CONSIDERATIONS_100000-499999  \
0                                   0                                     0   
1                                   0                                     0   
2                                   0                                     0   
3                                   1                                     0   
4                                   0                                     1   

   SPECIAL_CONSIDERATIONS_10M-50M  SPECIAL_CONSIDERATIONS_1M-5M  \
0                               0                             0   
1                               0                             0   
2                               0                             0   
3                               0                             0   
4                               0                             0   

   SPECIAL_CONSIDERATIONS_25000-99999  SPECIAL_CONSIDERATIONS_50M+  \
0                                   0                            0   
1                                   0                            0   
2                                   0                            0   
3                                   0                            0   
4                                   0                            0   

   SPECIAL_CONSIDERATIONS_5M-10M  INCOME_AMT_N  INCOME_AMT_Y  
0                              0             1             0  
1                              0             1             0  
2                              0             1             0  
3                              0             1             0  
4                              0             1             0  

[5 rows x 38 columns]

In [48]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = new_df.drop(["IS_SUCCESSFUL"], axis="columns").values
y=  new_df["IS_SUCCESSFUL"].values
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [49]:
print (X.shape)

(34299, 37)


In [50]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [51]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=111, activation="relu", input_dim=37))
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=74, activation="relu"))
nn.add(tf.keras.layers.Dense(units=37, activation="sigmoid"))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 111)               4218      
                                                                 
 dense_9 (Dense)             (None, 74)                8288      
                                                                 
 dense_10 (Dense)            (None, 37)                2775      
                                                                 
 dense_11 (Dense)            (None, 1)                 38        
                                                                 
Total params: 15319 (59.84 KB)
Trainable params: 15319 (59.84 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [53]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5729 - accuracy: 0.7199
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5612 - accuracy: 0.7267
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5596 - accuracy: 0.7293
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5580 - accuracy: 0.7283
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7298
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7298
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5550 - accuracy: 0.7306
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7325
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7320
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5528 - accura

In [54]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5706 - accuracy: 0.7251 - 467ms/epoch - 2ms/step
Loss: 0.5706362128257751, Accuracy: 0.7251312136650085


In [57]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save('AlphabetSoupCharity_optimization.keras')